In [7]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime

# Import ds_utils cleaning functions
from ds_utils.cleaning import (
    clean_german_numbers,
    parse_german_dates,
    clean_column_names,
    detect_outliers,
    remove_outliers,
    missing_value_report,
    fill_missing_values,
    convert_dtypes,
    normalize_text,
    register_janitor_methods,
)

from ds_utils.cleaning import (
    validate_dataframe,
    create_schema,
    create_numeric_schema,
    create_string_schema,
    create_email_schema,
    create_german_phone_schema,
    create_german_postal_code_schema,
    data_quality_report,
)

# Register custom janitor methods
register_janitor_methods()

## 1. Sample Data with German Formatting

In [5]:
# Create sample data with German number and date formatting
df_german = pd.DataFrame({
    'Produkt Name': ['Widget A', 'Widget B', 'Widget C', 'Widget D', 'Widget E'],
    'Preis (EUR)': ['1.234,56', '987,65', '2.345,00', '456,78', '12.345,67'],
    'Menge': ['1.000', '500', '2.500', '100', '10.000'],
    'Datum': ['15.03.2024', '22.01.2024', '08.12.2023', '30.06.2024', '01.09.2023'],
    'Umsatz': ['1.234.560,00', '493.825,00', '5.862.500,00', '45.678,00', '123.456.700,00'],
})

print("Original German-formatted data:")
display(df_german)

Original German-formatted data:


,Produkt Name,Preis (EUR),Menge,Datum,Umsatz
0,Widget A,"1.234,56",1.000,15.03.2024,"1.234.560,00"
1,Widget B,"987,65",500,22.01.2024,"493.825,00"
2,Widget C,"2.345,00",2.500,08.12.2023,"5.862.500,00"
3,Widget D,"456,78",100,30.06.2024,"45.678,00"
4,Widget E,"12.345,67",10.000,01.09.2023,"123.456.700,00"


## 2. Clean German Numbers

In [10]:
# Clean German number formats
df_cleaned = df_german.copy()

# Convert German number format to float
df_cleaned['preis_clean'] = clean_german_numbers(df_cleaned['Preis (EUR)'])
df_cleaned['menge_clean'] = clean_german_numbers(df_cleaned['Menge'])
df_cleaned['umsatz_clean'] = clean_german_numbers(df_cleaned['Umsatz'])

print("Cleaned numeric columns:")
display(df_cleaned[['Preis (EUR)', 'preis_clean', 'Menge', 'menge_clean', 'Umsatz', 'umsatz_clean']])

Cleaned numeric columns:


,Preis (EUR),preis_clean,Menge,menge_clean,Umsatz,umsatz_clean
0,"1.234,56",1234.56,1.000,1000,"1.234.560,00",1234560.0
1,"987,65",987.65,500,500,"493.825,00",493825.0
2,"2.345,00",2345.00,2.500,2500,"5.862.500,00",5862500.0
3,"456,78",456.78,100,100,"45.678,00",45678.0
4,"12.345,67",12345.67,10.000,10000,"123.456.700,00",123456700.0


## 3. Parse German Dates

In [11]:
# Parse German date format
df_cleaned['datum_clean'] = parse_german_dates(df_cleaned['Datum'])

print("Cleaned date column:")
display(df_cleaned[['Datum', 'datum_clean']])
print(f"\nData type: {df_cleaned['datum_clean'].dtype}")

Cleaned date column:


,Datum,datum_clean
0,15.03.2024,2024-03-15
1,22.01.2024,2024-01-22
2,08.12.2023,2023-12-08
3,30.06.2024,2024-06-30
4,01.09.2023,2023-09-01



Data type: datetime64[ns]


## 4. Clean Column Names

In [ ]:
# Create DataFrame with messy column names
df_messy = pd.DataFrame({
    'Product Name  ': [1, 2, 3],
    ' Sales (USD)': [100, 200, 300],
    'Quantity-Sold': [10, 20, 30],
    'Date Of Purchase': ['2024-01-01', '2024-01-02', '2024-01-03'],
    'Customer ID#': ['C001', 'C002', 'C003'],
})

print("Original columns:")
print(df_messy.columns.tolist())

# Clean column names
df_clean_names = clean_column_names(df_messy)

print("\nCleaned columns:")
print(df_clean_names.columns.tolist())

## 5. Outlier Detection and Removal

In [15]:
# Create data with outliers
np.random.seed(42)
df_outliers = pd.DataFrame({
    'value_a': np.concatenate([np.random.normal(50, 10, 95), [150, 200, -50, 180, 250]]),
    'value_b': np.concatenate([np.random.normal(100, 20, 95), [300, 400, -100, 350, 500]]),
})

print(f"Original data shape: {df_outliers.shape}")
print(f"Value A range: {df_outliers['value_a'].min():.2f} to {df_outliers['value_a'].max():.2f}")
print(f"Value B range: {df_outliers['value_b'].min():.2f} to {df_outliers['value_b'].max():.2f}")

Original data shape: (100, 2)
Value A range: -50.00 to 250.00
Value B range: -100.00 to 500.00


In [18]:
# Detect outliers using IQR method
outlier_mask = detect_outliers(df_outliers, columns=['value_a', 'value_b'], method='iqr')

print(f"Outliers detected:\n{outlier_mask.sum()}")
print("\nOutlier rows:")
display(df_outliers[outlier_mask.any(axis=1)])

Outliers detected:
value_a_outlier    6
value_b_outlier    5
dtype: int64

Outlier rows:


,value_a,value_b
74,23.802549,84.925277
95,150.000000,300.000000
96,200.000000,400.000000
97,-50.000000,-100.000000
98,180.000000,350.000000
99,250.000000,500.000000


In [19]:
# Remove outliers
df_no_outliers = remove_outliers(df_outliers, columns=['value_a', 'value_b'], method='iqr')

print(f"Data shape after outlier removal: {df_no_outliers.shape}")
print(f"Value A range: {df_no_outliers['value_a'].min():.2f} to {df_no_outliers['value_a'].max():.2f}")
print(f"Value B range: {df_no_outliers['value_b'].min():.2f} to {df_no_outliers['value_b'].max():.2f}")

Data shape after outlier removal: (94, 2)
Value A range: 30.12 to 68.52
Value B range: 61.62 to 154.40


## 6. Missing Value Handling

In [20]:
# Create data with missing values
np.random.seed(42)
df_missing = pd.DataFrame({
    'name': ['Alice', 'Bob', None, 'David', 'Eve', None, 'Grace', 'Henry'],
    'age': [25, np.nan, 35, 40, np.nan, 28, 45, np.nan],
    'salary': [50000, 60000, np.nan, 75000, 55000, np.nan, 80000, 65000],
    'department': ['Sales', 'IT', 'HR', None, 'Sales', 'IT', None, 'HR'],
})

print("Data with missing values:")
display(df_missing)

Data with missing values:


,name,age,salary,department
0,Alice,25.0,50000.0,Sales
1,Bob,NaN,60000.0,IT
2,None,35.0,NaN,HR
3,David,40.0,75000.0,None
4,Eve,NaN,55000.0,Sales
5,None,28.0,NaN,IT
6,Grace,45.0,80000.0,None
7,Henry,NaN,65000.0,HR


In [ ]:
# Generate missing value report
report = missing_value_report(df_missing)
print("Missing Value Report:")
display(report)

In [24]:
# Fill missing values with different strategies
df_filled = fill_missing_values(
    df_missing,
    strategies={
        'name': 'constant',
        'age': 'median',
        'salary': 'mean',
        'department': 'mode',
    },
    fill_values={'name': 'Unknown'},
)

print("Data after filling missing values:")
display(df_filled)

Data after filling missing values:


,name,age,salary,department
0,Alice,25.0,50000.000000,Sales
1,Bob,35.0,60000.000000,IT
2,Unknown,35.0,64166.666667,HR
3,David,40.0,75000.000000,HR
4,Eve,35.0,55000.000000,Sales
5,Unknown,28.0,64166.666667,IT
6,Grace,45.0,80000.000000,HR
7,Henry,35.0,65000.000000,HR


## 7. Text Normalization

In [13]:
# Create data with messy text
df_text = pd.DataFrame({
    'name': ['  JOHN DOE  ', 'jane   smith', 'BOB JOHNSON', ' alice williams '],
    'city': ['NEW YORK', 'los angeles', 'CHICAGO', 'san francisco'],
    'email': ['JOHN@EMAIL.COM', 'jane@email.com', 'BOB@EMAIL.COM', 'alice@email.com'],
})

print("Original text data:")
display(df_text)

Original text data:


,name,city,email
0,JOHN DOE,NEW YORK,JOHN@EMAIL.COM
1,jane smith,los angeles,jane@email.com
2,BOB JOHNSON,CHICAGO,BOB@EMAIL.COM
3,alice williams,san francisco,alice@email.com


In [14]:
# Normalize text columns
df_text_clean = df_text.copy()
df_text_clean['name'] = normalize_text(df_text['name'], case='title')
df_text_clean['city'] = normalize_text(df_text['city'], case='title')
df_text_clean['email'] = normalize_text(df_text['email'], case='lower')

print("Normalized text data:")
display(df_text_clean)

Normalized text data:


,name,city,email
0,John Doe,New York,john@email.com
1,Jane Smith,Los Angeles,jane@email.com
2,Bob Johnson,Chicago,bob@email.com
3,Alice Williams,San Francisco,alice@email.com


## 8. Data Type Conversion

In [ ]:
# Create data with string types that should be converted
df_types = pd.DataFrame({
    'id': ['1', '2', '3', '4', '5'],
    'amount': ['100.50', '200.75', '300.25', '400.00', '500.50'],
    'is_active': ['True', 'False', 'True', 'True', 'False'],
    'date': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05'],
    'category': ['A', 'B', 'A', 'C', 'B'],
})

print("Original data types:")
print(df_types.dtypes)
print()

In [ ]:
# Convert data types
df_converted = convert_dtypes(
    df_types,
    type_map={
        'id': 'int',
        'amount': 'float',
        'is_active': 'bool',
        'date': 'datetime',
        'category': 'category',
    }
)

print("Converted data types:")
print(df_converted.dtypes)

## 9. Data Validation with Pandera

In [ ]:
# Create a schema for validation
import pandera as pa

# Define schema using helper functions
age_col = create_numeric_schema(min_value=0, max_value=120, nullable=False)
email_col = create_email_schema(nullable=False)
postal_col = create_german_postal_code_schema(nullable=True)

schema = pa.DataFrameSchema({
    'name': pa.Column(str, pa.Check.str_length(min_value=1, max_value=100)),
    'age': age_col,
    'email': email_col,
    'postal_code': postal_col,
})

print("Schema created for validation")

In [ ]:
# Create valid data
df_valid = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [25, 30, 35],
    'email': ['alice@example.com', 'bob@example.com', 'charlie@example.com'],
    'postal_code': ['12345', '67890', None],
})

# Validate the data
is_valid, errors = validate_dataframe(df_valid, schema)
print(f"Data is valid: {is_valid}")
if not is_valid:
    print(f"Errors: {errors}")

In [ ]:
# Create invalid data
df_invalid = pd.DataFrame({
    'name': ['Alice', '', 'Charlie'],  # Empty name
    'age': [25, -5, 150],  # Negative and too high age
    'email': ['alice@example.com', 'not-an-email', 'charlie@example.com'],  # Invalid email
    'postal_code': ['12345', 'ABCDE', '123'],  # Invalid postal codes
})

# Validate the data
is_valid, errors = validate_dataframe(df_invalid, schema)
print(f"Data is valid: {is_valid}")
if not is_valid:
    print(f"\nValidation errors:\n{errors}")

## 10. Data Quality Report

In [ ]:
# Create sample data for quality analysis
np.random.seed(42)
df_quality = pd.DataFrame({
    'id': range(1, 101),
    'name': ['Person ' + str(i) if i % 10 != 0 else None for i in range(1, 101)],
    'age': [np.random.randint(18, 80) if i % 7 != 0 else np.nan for i in range(1, 101)],
    'salary': [np.random.uniform(30000, 100000) if i % 5 != 0 else np.nan for i in range(1, 101)],
    'department': np.random.choice(['Sales', 'IT', 'HR', 'Finance', 'Marketing'], 100),
})

print(f"Sample data shape: {df_quality.shape}")
display(df_quality.head())

In [ ]:
# Generate comprehensive data quality report
quality_report = data_quality_report(df_quality)
print("Data Quality Report:")
display(quality_report)